In [1]:
import pandas as pd
import requests, os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def downloadCSV():
    n, m = 15, 16
    url = 'https://dati.istruzione.it/opendata'
    namef = '/ALUCORSOETASTA20{}{}20{}0831.csv'
    lsfile = {}
    for i in range(6):
        fname = namef.format(n, m, m)
        lsfile['./static/data'+fname] = 'OK'
        try:
            f = open('./static/data'+fname, 'x')
            f.close()
        except:
            pass
        if os.path.getsize('./static/data'+fname)==0:
            f = open('./static/data'+fname, 'wb')
            response = requests.get(url+fname)
            if response.reason == 'OK':
                f.write(response.content)
            lsfile['./static/data'+fname] = response.reason
            f.close()
        n += 1
        m += 1
    return lsfile

In [3]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [4]:
def toNum(s):
    if '<' in s:
        return int(s[-7:-5])-1
    elif '>' in s:
        return int(s[-7:-5])-1
    else:
        return int(s[-7:-5])
            

In [5]:
def getTablesResults(data):#, cod_mecc):
    tbls = []
    for i in [path for path, status in data.items() if status == 'OK']:
        tab = pd.read_csv(i)
        tbls.append(tab)
    tab = pd.concat(tbls, ignore_index=True)
    #print(tab)
    tab['ANNOSCOLASTICO'] = tab['ANNOSCOLASTICO'].apply(lambda n : str(n)[0:-2]+'/'+str(n)[-2:])
    tab = tab.loc[tab['FASCIAETA']!='Non Classificabile']
    tab['FASCIAETA'] = tab['FASCIAETA'].apply(toNum)
    tab = tab.groupby(['CODICESCUOLA', 'ANNOSCOLASTICO', 'ORDINESCUOLA', 'ANNOCORSO', 'FASCIAETA']).agg({'ALUNNI': 'sum'})
    #tab2 = tab.groupby(['CODICESCUOLA', 'ANNOSCOLASTICO', 'ORDINESCUOLA', 'ANNOCORSO']).agg({'ALUNNI': 'sum'})
    return tab

In [119]:
def getTableByCodeTot(tab, cod):
    t = tab.loc[cod].reset_index()
    t = t.groupby(['ANNOSCOLASTICO', 'ANNOCORSO']).agg({'ALUNNI': 'sum'})
    t = t.reset_index()
    t = t.pivot(index='ANNOCORSO', columns='ANNOSCOLASTICO', values='ALUNNI')
    t = pd.DataFrame(t.to_records())
    t.rename(columns={'ANNOCORSO':'Anno di corso'}, inplace=True)
    t = t.set_index('Anno di corso')
    t.to_csv('./static/results/{}{}.csv'.format(cod, o))
    return t

In [90]:
tab = getTablesResults(downloadCSV())
sims = sorted([c for c in list(set(tab.index.get_level_values(0))) if similar(c, 'MITF03801E')>0.75], reverse=True, key = lambda c : similar(c, 'MITF03801E'))

In [124]:
def getTableByCode(tab, cod, collect_regular=True):
    tb = tab.loc[cod].reset_index()
    for o in set(tb.ORDINESCUOLA):
        t = tb[(tb.FASCIAETA<=tb.ANNOCORSO+regularage[o])==collect_regular]
        t = t.groupby(['ANNOSCOLASTICO', 'ANNOCORSO']).agg({'ALUNNI': 'sum'})
        t = t.reset_index()
        t = t.pivot(index='ANNOCORSO', columns='ANNOSCOLASTICO', values='ALUNNI')
        t = pd.DataFrame(t.to_records())
        t.rename(columns={'ANNOCORSO':'Anno di corso'}, inplace=True)
        t = t.set_index('Anno di corso')
        t.to_csv('./static/results/{}{}{}.csv'.format(cod, o, collect_regular))
        return t

In [66]:
regularage = {
    'SCUOLA PRIMARIA' : 5,
    'SCUOLA SECONDARIA I GRADO' : 10,
    'SCUOLA SECONDARIA II GRADO' : 13
}

In [125]:
table = getTableByCode(tab, 'MITF03801E')

In [126]:
table1 = getTableByCode(tab, 'MITF03801E', collect_regular=False)